<img src="docs/banner.png" alt="Deparatemento de Ingeniería de Sistemas y Computación, Universidad de los Andes">

# Introducción al Open Data Cube

**Introducción**

El Open Data Cube es un conjunto de librerías que facilitan el proceso de **Organización**, **Consulta** y **Recuperación** de información de imágenes de satélite. En la presente practica relizaremos el proceso de consulta y análisis de una imágen satelital mediante las funcionalidades que ofrece el ODC.

**Contenido**

1. importar librerías
2. Consulta del área de estudio
3. Características de la imágen obtenida
4. Aplicación de un algoritmo de análisis
5. Visualización de resultados

## 1. Importar librerías

En esta sección se importan las librerías cuya funicionalidades particulares son requeridas.

In [ ]:
# las funcionalidades del open data cube son accedidas 
# por medio de la librería datacube
import datacube

# Librería usada para la carga de polígonos
import geopandas as gpd

# Librería usada para visualización de datos
import matplotlib.pyplot as plt

# Desactiva los warnings en el notebook
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Configuración de Drivers para leer polígonos en formato KMLs
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

## 2. Consulta del área de estudio

(Opción 1) Consultar un área a partir de un polígono

In [ ]:
# Carga del archivo .kml
df_polygon = gpd.read_file("1.kml",driver='KML')
df_polygon = df_polygon.to_crs('EPSG:32719')

# Pintar el polígono seleccionado
fig, ax = plt.subplots(figsize=(5,5))
geo_df_predio.boundary.plot(ax=ax,color='red')

# Obtención de la geometría del polígono del GeoDataFrame
geometry_predio = geo_df_predio['geometry'][0]

# Obtención de los límites del cuadrado que enmarca el polígono
minx, miny, maxx, maxy = geometry_predio.bounds

# Aumento del aŕea del cuadrado para "EPSG:32719"
# 2 kilómetros
buffer = 2000

minx = minx - buffer
miny = miny - buffer
maxx = maxx + buffer
maxy = maxy + buffer

# Parámetros de área a ser consultada
set_study_area_lat = (miny,maxy)
set_study_area_lon = (minx,maxx)

print(set_study_area_lat)
print(set_study_area_lon)

(Opción 2) Consultar un área a partir de un rango de coordenadas

In [ ]:
# # Parámetros de área a ser consultada
# set_study_area_lat = (,)
# set_study_area_lon = (,)

(Opción 3) Consultar un área a partir de un punto

In [ ]:
# # Definición de las coordenadas del punto
# central_lat = 5.672302
# central_lon = -73.257158

# # Aumento del aŕea del cuadrado para "EPSG:32719"
# # 2 kilómetros
# buffer = 2000

# # Calculo del cuadro delimitador (bounding box) para el área de estudio
# set_study_area_lat = (central_lat - buffer, central_lat + buffer)
# set_study_area_lon = (central_lon - buffer, central_lon + buffer)

# print(set_study_area_lat)
# print(set_study_area_lon)

Configuración parámetros adicionales de la consulta

In [ ]:
# Seleeción del producto
set_product = "s2_sen2cor_ard_granule_EO3"

# Selección del periodo de tiempo
set_time = ('2020-04-10', '2020-04-18')

# Selección de las bandas
set_measurements = [
    "red",
    "blue",
    "green",
    "nir",
    "swir1",
    "swir2",
    "scl"
]

# Sistema de coordenadas para especificar latitud y longitud
set_input_crs = 'EPSG:32719'

# Sistema de coordenada de salida del dataset.
set_output_crs = 'EPSG:32719'

# Resolución 
set_resolution = (-10, 10)

Carga de información en el Open Data Cube

In [ ]:
dc = datacube.Datacube(app="Cana")

dataset = dc.load(
    product=set_product,
    x=set_study_area_lon,
    y=set_study_area_lat,
    time=set_time,
    measurements=set_measurements,
    crs=set_input_crs,
    output_crs=set_output_crs,
    resolution=set_resolution,
)

dataset = dataset.assign(**additional_indices)
dataset

## 3. Características de la imágen obtenida

## 4. Aplicación de un algoritmo de análisis

In [ ]:
def savi(dataset):
    """
    Soil Adjusted Vegetation Index
    Referencia: https://www.usgs.gov/core-science-systems/nli/landsat/landsat-soil-adjusted-vegetation-index
    """
    return ((dataset.nir - dataset.red)/(dataset.nir + dataset.red + 0.5))*(1.5)

additional_indices = {
    'savi':savi,
}

## 5. Visualización de resultados

In [ ]:
rgb = dataset[["red","green","blue"]].to_array(dim='color')
rgb = rgb.transpose(*(rgb.dims[1:]+rgb.dims[:1]))  # make 'color' the last dimension
img = rgb.plot.imshow(col='time',col_wrap=4,add_colorbar=False,vmin=0,vmax=1500)

for axes in img.axes.flat:
    geo_df_predio.boundary.plot(ax=axes,markersize=20,color='blue',marker='o')
plt.show()